In [1]:
import joblib 
import numpy as np
import pandas as pd 
from pathlib import Path
import matplotlib.pyplot as plt
import hist
from hist import Hist 
from uncertainties import ufloat, unumpy
import os, sys
sys.path.append("../core")

from utils import *

In [2]:
#### Load reweighting file 
reweight_file_path = '/global/cfs/projectdirs/atlas/hrzhao/HEP_Repo/QG_Calibration/NewWorkflow/test_reweight/test_reweight_factor.pkl'
reweight_file_path = Path(reweight_file_path)
reweight_factor = joblib.load(reweight_file_path)


In [5]:
#### Define reweighting options 
reweighting_vars = ['jet_nTracks', 'jet_trackBDT', 'GBDT_newScore'] 
all_weight_options = ['event_weight'] + [f'{reweight_var}_{parton}_reweighting_weights' for reweight_var in reweighting_vars for parton in ['quark', 'gluon']]

In [6]:
for period_mc in ["A", "D", "E"]:
    pythia_path = f'/global/cfs/projectdirs/atlas/hrzhao/HEP_Repo/QG_Calibration/tmp/pythia{period_mc}_pred.pkl'
    pythia_pd = joblib.load(pythia_path)
    pythia_pd = pythia_pd[(pythia_pd["jet_nTracks"] > 1) & (pythia_pd["target"] != '-')] 

    reweighted_pythia_pd = attach_reweight_factor(pythia_pd, reweight_factor)
    joblib.dump(reweighted_pythia_pd, f'pythia{period_mc}_reweighted.pkl')

    pythia_hists = {}
    for weight in all_weight_options:
        pythia_hists[weight] = digitize_pd(reweighted_pythia_pd, weight, data_type="MC")

    joblib.dump(pythia_hists, f"pythia{period_mc}_hists.pkl")

In [7]:
for period_data in ["1516", "17", "18"]:
    data_path = f'/global/cfs/projectdirs/atlas/hrzhao/HEP_Repo/QG_Calibration/tmp/data{period_data}_pred.pkl'
    data_pd = joblib.load(data_path)
    data_pd = data_pd[(data_pd["jet_nTracks"] > 1)] 

    reweighted_data_pd = attach_reweight_factor(data_pd, reweight_factor)
    joblib.dump(reweighted_data_pd, f'data{period_data}_reweighted.pkl')

    data_hists = {}

    for weight in all_weight_options:
        data_hists[weight] = digitize_pd(reweighted_data_pd, weight, data_type= 'Data')
        
    
    joblib.dump(data_hists, f"data{period_data}_hists.pkl")

In [ ]:
def merge_unumpy_pkls():
    